In [24]:
import sys
sys.path.insert(0, '/home/josiah/Desktop/Work/Caltech/SEDM/')
from apo_observing_functions import *
import os
import argparse
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.time import Time

In [2]:
'''
directly copy output from 
https://www.projectpluto.com/ephem.htm
 for obvservables using:

Number of steps : 2160
Step size: 0.00138888 (2 minutes)
Separate RA and dec


pbpaste > project_pluto_test; python SEDM_ephemeris_generator.py -if project_pluto_test -on  C2019Q4 > C2019Q4_ephemerides_SEDM.csv

input is like:

Date (UTC).dddddddd   RA              Dec         delta   r     elong  mag  RA '/hr dec  " sig PA
---- --------------  --------------  -----------  -----  ------ -----  --- ------------- --------
2019 08 16.07225723  02 36 23.275   +04 00 07.47  .78944 1.4306 104.4 20.5   1.32   0.01 1.64 125

output format needs to be in the following format:

objname, ra(degrees, 6 digits), dec(degrees, 6 digits), equinox 6 digits, ra_rate(arcsec/hr, 6 digits), dec_rate(arcsec/hr, 6 digits), V_mag 6 digits
5879, 240.058320, 30.052050, 2000.000000, -36.952900, -44.958300, 2019-Jun-04-04:12:32.540, 19.250000


python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTFDD01 > ZTFDD01_ephemerides_SEDM.csv

#ZTF0G6I
python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0G6I > ZTF0G6I_ephemerides_SEDM.csv

#ZTF0GRO
python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0GRO > ZTF0GRO_ephemerides_SEDM.csv

#ZTF0Ion
python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0Ion > ZTF0Ion_ephemerides_SEDM.csv


#sample executions
pbpaste > project_pluto_test; python SEDM_ephemeris_generator.py -if project_pluto_test -on  C2019Q4 > C2019Q4_ephemerides_SEDM.csv

pbpaste > project_pluto_test; 
python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_SEDM_latest_obs_up_to_2020_01_11_v4.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  P10Z5vx > P10Z5vx_ephemerides_SEDM.csv

python SEDM_ephemeris_generator.py -if ZTF0IgE_ephemerides -on  ZTF0IgE > ZTF0IgE_ephemerides_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0KFX > ZTF0KFX_ephemerides_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0KcP > ZTF0KcP_ephemerides_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0Kij > ZTF0Kij_ephemerides_SEDM.csv

#2020 AV2 2021 August apparition
python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_2021_August_10_to_13_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_2021_August_13_to_16_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_2021_August_16_to_19_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_2021_August_19_to_22_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_2021_August_26_to_29_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2020AV2 > 2020AV2_ephemerides_2021_August_26_to_29_SEDM.csv

python SEDM_ephemeris_generator.py -if project_pluto_test -on  2021BA > 2021BA_ephemerides_2022_January_7_SEDM.csv

#ZTF0Nf7
python SEDM_ephemeris_generator.py -if project_pluto_test -on  ZTF0Nf7 > ZTF0Nf7_ephemerides_2022_Marh_20_SEDM.csv

''';

In [116]:
#parser = argparse.ArgumentParser()
#parser.add_argument("-if", "--infile", 
#                    help="location of input ephemeris from Project Pluto e.g., /home/josiah/Desktop/Work/Caltech/SEDM/project_pluto_test")
#parser.add_argument("-on", "--object_name", help="object name, e.g., ZTF05iT")
#args = parser.parse_args()

#infile = str(args.infile)
#object_name = str(args.object_name)
infile = 'project_pluto_test'
object_name = 'C_2022_E3'

equinox= 2000.0
space = ' '
dash = '-'

os.system('sed \'/----/,$!d\' ' + infile + ' | tail -n+2 > ' + object_name +'_tmp')
year,month,date,RA_H,RA_M,RA_S,Dec_d,Dec_m,Dec_s,V_mag,ra_rate_arcmin_hr,dec_rate_arcmin_hr = np.genfromtxt(infile ,usecols =(0,1,2,3,4,5,6,7,8,12,13,14)).T


os.system('rm ' + object_name +'_tmp')

coords = []
for k in range(len(RA_H)):
    coords.append(str(int(RA_H[k])) + ':' + str(int(RA_M[k])) + ':' + str(RA_S[k]) + space + str(int(Dec_d[k])) + 
                  ':' + str(int(Dec_m[k]))+ ':' + str(Dec_s[k]))
RA_deg = SkyCoord(coords, unit='deg').ra.degree
Dec_deg = SkyCoord(coords, unit='deg').dec.degree
ra_rate_arcsec_hr, dec_rate_arcsec_hr =ra_rate_arcmin_hr*60., dec_rate_arcmin_hr*60.#arcmin_hr to arcsec_hr

string_mjds = []
for i in range(len(year)):
    string_mjds.append(str(int(year[i]))+'-'+str(int(month[i]))+'-'+str(int(date[i])))

time_mjds= []
for j in string_mjds:
    time_mjds.append(Time(j).mjd)

date_mjd_UTC = time_mjds + np.modf(date)[0]

t = Time(date_mjd_UTC, format='mjd', scale='utc')
t_isot = t.isot

entire_date_string = np.copy(date_mjd_UTC.astype('U32'))


for i in range(0,len(entire_date_string)):
    t_date_storage = t.isot[i][:t.isot[i].find('T')]
    year_storage = t_date_storage[:t_date_storage.find('-')]
    month_storage = dictionary_month_string_two_digit(t_date_storage[t_date_storage.find('-')+1: t_date_storage.find('-')+3])
    day_storage = t_date_storage[t_date_storage.find('-')+4:]
    time_storage = t_isot[i][t_isot[i].find('T')+1:]
    entire_date_string[i] = year_storage + dash + month_storage + dash + day_storage + space + time_storage
    print('{}, {}, {}, {}, {}, {}, {}, {}'.format(object_name, RA_deg[i] , Dec_deg[i] , equinox , 
                                                  ra_rate_arcsec_hr[i] , dec_rate_arcsec_hr[i] , 
                                                  entire_date_string[i], V_mag[i]))

#times_string_isot_no_t = np.array([s.replace('T','-') for s in t.isot])
#print 'objname, ra(degrees), dec(degrees), equinox, ra_rate(arcsec/hr), dec_rate(arcsec/hr), time, V'
print('objname, ra(degrees), dec(degrees), equinox, ra_rate(arcsec/hr), dec_rate(arcsec/hr), time, V')

C_2022_E3, 18.447544444444446, 34.50215277777778, 2000.0, -87.36, 24.66, 2022-Jul-13 21:26:34.723, 14.6
C_2022_E3, 18.447479166666668, 34.502380555555554, 2000.0, -87.36, 24.66, 2022-Jul-13 21:28:34.722, 14.6
C_2022_E3, 18.447413611111113, 34.502608333333335, 2000.0, -87.36, 24.66, 2022-Jul-13 21:30:34.721, 14.6
C_2022_E3, 18.447348333333334, 34.50283888888889, 2000.0, -87.42, 24.66, 2022-Jul-13 21:32:34.721, 14.6
C_2022_E3, 18.44728277777778, 34.50306666666667, 2000.0, -87.42, 24.66, 2022-Jul-13 21:34:34.720, 14.6
C_2022_E3, 18.44721722222222, 34.50329722222222, 2000.0, -87.42, 24.66, 2022-Jul-13 21:36:34.718, 14.6
C_2022_E3, 18.447151944444446, 34.503525, 2000.0, -87.42, 24.66, 2022-Jul-13 21:38:34.718, 14.6
C_2022_E3, 18.447086388888888, 34.50375277777778, 2000.0, -87.42, 24.66, 2022-Jul-13 21:40:34.717, 14.6
C_2022_E3, 18.447020833333333, 34.50398333333333, 2000.0, -87.42, 24.66, 2022-Jul-13 21:42:34.717, 14.6
C_2022_E3, 18.446955555555554, 34.50421111111111, 2000.0, -87.42, 24.66,

In [91]:
RA_deg = np.array([(map(convert_h_m_s_to_deg,RA_H,RA_M,RA_S))]).T[0]
RA_deg

In [110]:
coords = []
for k in range(len(RA_H)):
    coords.append(str(int(RA_H[k])) + ':' + str(int(RA_M[k])) + ':' + str(RA_S[k]) + space + str(int(Dec_d[k])) + 
                  ':' + str(int(Dec_m[k]))+ ':' + str(Dec_s[k]))

In [112]:
dug = SkyCoord(coords, unit='deg').ra.degree
dug

array([18.44754444, 18.44747917, 18.44741361, ..., 18.30404139,
       18.303975  , 18.30390833])

In [63]:
y = []
for i in range(len(year)):
    y.append(str(int(year[i]))+'-'+str(int(month[i]))+'-'+str(int(date[i])))

In [78]:
f = []
for x in y:
    f.append(Time(x).mjd)
tot = f + np.modf(date)[0]
tot

array([59773.89345744, 59773.89484632, 59773.8962352 , ...,
       59776.8892716 , 59776.89066048, 59776.89204936])

In [83]:
tot.astype('U32')

array(['59773.89345744', '59773.89484632', '59773.8962352', ...,
       '59776.8892716', '59776.89066048', '59776.89204936'], dtype='<U32')